In [30]:
!pip install gensim==3.8.3
!pip install keras --upgrade
!pip install pandas --upgrade
!pip install tensorflow --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

In [32]:
vocab_size = 32965
# WORD2VEC 
W2V_SIZE = 300
SEQUENCE_LENGTH = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

In [33]:
# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 10
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#LOADING
load_dir = '/content/drive/MyDrive/nns/'
train_test_dir = load_dir+'saved_train_test/'

embedding_matrix = np.load(load_dir+'embedding_matrix.npy')

x_train = np.load(train_test_dir+'x_train.npy')
y_train = np.load(train_test_dir+'y_train.npy')

x_test = np.load(train_test_dir+'x_test.npy')
y_test = np.load(train_test_dir+'y_test.npy')

In [36]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (40000, 300)
y_train (40000, 1)

x_test (10000, 300)
y_test (10000, 1)


###NN Model

In [37]:
#HYPER PARAMETERS
model_name = "LSTM"
num_epochs = 10
batch_size = 1024
learning_rate = 1e-5
lstm_units = 256
momentum=.9
sequence_length=300
activation="sigmoid"
optimizer='adam'

In [38]:
#MODEL
embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], 
                            input_length=SEQUENCE_LENGTH,  
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.LSTM(lstm_units,
                               recurrent_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))


model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 300, 300)          9889500   
                                                                 
 dropout_5 (Dropout)         (None, 300, 300)          0         
                                                                 
 lstm_5 (LSTM)               (None, 256)               570368    
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 10,460,125
Trainable params: 570,625
Non-trainable params: 9,889,500
_________________________________________________________________


In [39]:
#OPTIMIZATION
callbacks = [ tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
#TRAINING
%%time
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/10
36/36 [==============================] - 16s 398ms/step - loss: 0.5802 - accuracy: 0.6926 - val_loss: 0.5428 - val_accuracy: 0.7203 - lr: 0.0010
Epoch 2/10
36/36 [==============================] - 14s 382ms/step - loss: 0.5481 - accuracy: 0.7193 - val_loss: 0.5355 - val_accuracy: 0.7245 - lr: 0.0010
Epoch 3/10
36/36 [==============================] - 14s 383ms/step - loss: 0.5424 - accuracy: 0.7221 - val_loss: 0.5381 - val_accuracy: 0.7172 - lr: 0.0010
Epoch 4/10
36/36 [==============================] - 13s 372ms/step - loss: 0.5290 - accuracy: 0.7312 - val_loss: 0.5269 - val_accuracy: 0.7297 - lr: 0.0010
Epoch 5/10
36/36 [==============================] - 13s 361ms/step - loss: 0.5195 - accuracy: 0.7376 - val_loss: 0.5257 - val_accuracy: 0.7327 - lr: 0.0010
Epoch 6/10
36/36 [==============================] - 13s 360ms/step - loss: 0.5144 - accuracy: 0.7401 - val_loss: 0.5270 - val_accuracy: 0.7320 - lr: 0.0010
Epoch 7/10
36/36 [==============================] - 13s 363ms/st

In [41]:
#TESTING
score = model.evaluate(x_test, y_test, batch_size=batch_size)

print("ACCURACY:",score[1])
print("LOSS:",score[0])

acc = history.history['accuracy']
print("acc:", acc)
val_acc = history.history['val_accuracy']
print("val_acc:", val_acc)
loss = history.history['loss']
print("loss:", loss)
val_loss = history.history['val_loss']
print("val_loss:", val_loss)


10/10 [==============================] - 1s 115ms/step - loss: 0.5140 - accuracy: 0.7467
ACCURACY: 0.7466999888420105
LOSS: 0.5139989852905273
acc: [0.6925555467605591, 0.7192777991294861, 0.7221111059188843, 0.7312222123146057, 0.737583339214325, 0.7401388883590698, 0.7474444508552551, 0.7510833144187927, 0.7557500004768372, 0.757111132144928]
val_acc: [0.7202500104904175, 0.7245000004768372, 0.7172499895095825, 0.7297499775886536, 0.7327499985694885, 0.7319999933242798, 0.7315000295639038, 0.7394999861717224, 0.7357500195503235, 0.734250009059906]
loss: [0.5802288055419922, 0.548066258430481, 0.5424219965934753, 0.5290154218673706, 0.519485592842102, 0.5143701434135437, 0.5064927339553833, 0.5017595887184143, 0.4939129650592804, 0.48861241340637207]
val_loss: [0.542811393737793, 0.5355402231216431, 0.5380504131317139, 0.5268732309341431, 0.5256502628326416, 0.527008056640625, 0.5236207842826843, 0.5238733291625977, 0.5249074697494507, 0.5294645428657532]


In [42]:
#SAVING 
save_dir = '/content/drive/MyDrive/nns/saved_nn_models/'
model_name = "LSTM"
model.save(save_dir+model_name+".h5")


In [43]:
!ls /content/drive/MyDrive/nns/saved_nn_models/

FFNN1.h5  FFNN3.h5  FFNN5.h5  LSTM.h5  RNN.h5
